In [1]:
import face_recognition as fr

In [4]:
#face registration
#####################
import cv2
import pandas as pd
import face_recognition as fr
fname = 'features.csv'
try:
    df = pd.read_csv(fname)
except:    
    df = pd.DataFrame({'name':[],'enc':[]})# data creation

fd = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)
vid = cv2.VideoCapture(0)
counter = 0  
names = []
feats = []
name = input('Enter your name:')
while True:
    ack, img = vid.read()
    if ack:
        faces = fd.detectMultiScale(img, 1.2,10,minSize=(150,150))
        if len(faces) == 1:
            x,y,w,h = faces[0]
            face_img = img[y:y+h,x:x+w,:].copy()
            face_enc = fr.face_encodings(face_img)
            if len(face_enc) == 1:
                counter += 1
                names += [name]
                feats += [face_enc[0].tolist()]                
        
            if counter == 20:
                f = pd.DataFrame({'name':names,'enc':feats})
                df = pd.concat([df,f], axis=0, ignore_index=True)# merge two databases
                df.to_csv(fname)
                break 
        cv2.imshow('preview', img) #depends on requirement
        key = cv2.waitKey(1)
        if key == ord('x'):
            break
cv2.destroyAllWindows()
vid.release() 

In [7]:
# face recognition
import cv2
import numpy as np
import pandas as pd
import face_recognition as fr
fname = 'features.csv'
try:
    df = pd.read_csv(fname)
except:    
    print('Face Database not found. halt')
else: 
   fd = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)
vid = cv2.VideoCapture(0)
while True:
    ack, img = vid.read()
    if ack:
        faces = fd.detectMultiScale(img, 1.2,10,minSize=(150,150))
        if len(faces) == 1:
            x,y,w,h = faces[0]
            face_img = img[y:y+h,x:x+w,:].copy()
            face_enc = fr.face_encodings(face_img)
            if len(face_enc) == 1:
                 feats_data = df['enc'].apply(lambda x:eval(x)).values.tolist()
                 matches = fr.compare_faces(
                    face_enc, np.array(feats_data))
                 if True in matches:
                      match_ind = matches.index(True)
                      name = df['name'][match_ind]
                 else:
                    name = ('unknown')      
                 cv2.putText(img, name,(150,150),
                    cv2.FONT_HERSHEY_PLAIN, 5, (0,0,255),10
                    )
        cv2.imshow('preview', img) 
        key = cv2.waitKey(1)
        if key == ord('x'):
            break
cv2.destroyAllWindows()
vid.release() 
